Evaluate convergence of the Eyring-Kramers constants to the predicted $n\to \infty$ limits.

In [ ]:
using Plots
using Printf
using Random
using LaTeXStrings
using LinearAlgebra
using ForwardDiff

In [ ]:
default(xtickfontsize=14, ytickfontsize=14,
    guidefontsize=14, legendfontsize=12, lw=2, ms=8)

In [ ]:
push!(LOAD_PATH, "../TwistedStates");
push!(LOAD_PATH, "../ClassicalKuramoto");

In [ ]:
using TwistedStates
using ClassicalKuramoto

# Prefactor
$$
\begin{split}
C(q,n) &= \frac{1}{n}\frac{2\pi }{|\mu_1|}\sqrt{\frac{|\mu_1| \mu_2\cdots\mu_{n-1}}{\lambda_1\lambda_2\cdots \lambda_{n-1}}}\\
& = \frac{1}{K n }\frac{1}{\cos(2\pi \hat{q}/n)}\frac{1}{|\nu_1|}\left(\frac{\cos(2\pi \hat{q}/n)}{\cos(2\pi (q+1)/n)}\right)^{n/2}\sqrt{\frac{|\nu_1|\nu_2\cdots \nu_{n-1}}{\lambda_1^{\circ}\lambda_2^{\circ}\cdots \lambda_{n-1}^{\circ}}}
\end{split}
$$
where
$$
\begin{align}
\nu_1&= -\frac{4}{3} + \mathrm{O}(3^{-n})\\
\cos(2\pi \hat{q}/n) & = 1 +\mathrm{O}(n^{-2})\\
\left(\frac{\cos(2\pi \hat{q}/n)}{\cos(2\pi (q+1)/n)}\right)^{n/2} & = 1 + \frac{\pi^2(3+4q)}{4n} + \mathrm{O}(n^{-2})\\
\sqrt{\frac{|\nu_1|\nu_2\cdots \nu_{n-1}}{\lambda_1^{\circ}\lambda_2^{\circ}\cdots \lambda_{n-1}^{\circ}}}&  = \sqrt{1-2/n} = 1 - 1/n + \mathrm{O}(n^{-2})
\end{align}
$$

Thus, 
$$
n \cdot K \cdot C(q,n) = \frac{3}{4} \cdot \left[1 + \left(\frac{\pi^2(3+4q)}{4}-1\right) \frac{1}{n}\right]\cdot + \mathrm{O}(n^{-2})
$$

# Define functions
These are used for visualization

In [ ]:
k = 1;
α = 0;
E = u -> ClassicalKuramoto.energy(u, k, α);

# compute exact energy barrier
function ΔE(n,q)
    q_init = construct_q_twisted(n, q+1)
    q_saddle = construct_l_saddle(n, q)
    return E(q_saddle) - E(q_init)
end

# asymptotic formula
function ΔE_asympt(n,q)
    return 1/π - (q+3/4)*π/n
end

# use automatic differentiation for the Hessian
HessE(u) = ForwardDiff.hessian(E, u);

# compute exact scaled prefactor
function nC(n,q)
    q_init = construct_q_twisted(n, q+1)
    q_saddle = construct_l_saddle(n, q)

    evals_min = eigvals(HessE(q_init));
    nonzero_evals_min = evals_min[2:end];
    evals_saddle = eigvals(HessE(q_saddle));
    nonzero_evals_saddle = [evals_saddle[1]; evals_saddle[3:end]];
    C_ = (2 * π) / (abs(nonzero_evals_saddle[1])) * sqrt(prod(abs.(nonzero_evals_saddle) ./ (nonzero_evals_min)));
    return C_
end

# asymptotic formula
function nC_asympt(n, q)
    C_ = .75 * (1+ (π^2 * (4*q+3)-4)/(4*n))
    return C_
end



# Visualize
Plot several different cases

In [ ]:
q = 0;
n_vals = 5 * 2 .^ (2:1:8);
scatter(n_vals, [abs(ΔE(n_, q) - ΔE_asympt(n_, q)) for n_ in n_vals],
    xscale=:log10, yscale=:log10, label=L"$K^{-1}H_{q+1}$ Error")
scatter!(n_vals, [abs(nC(n_, q) - nC_asympt(n_, q)) for n_ in n_vals], 
    label=L"$nK C(q,n)$ Error", marker=:d)
plot!(n_vals, n_vals.^(-2),color=:black, ls=:dash, label=L"$\propto n^{-2}$")
xlabel!(L"n")
plot!(legend=:topright)
title!(latexstring(@sprintf("q = %d", q)))

In [ ]:
q = 1;
n_vals = 5 * 2 .^ (2:1:8);
scatter(n_vals, [abs(ΔE(n_, q) - ΔE_asympt(n_, q)) for n_ in n_vals],
    xscale=:log10, yscale=:log10, label=L"$K^{-1}H_{q+1}$ Error")
scatter!(n_vals, [abs(nC(n_, q) - nC_asympt(n_, q)) for n_ in n_vals], 
    label=L"$nK C(q,n)$ Error", marker=:d)
plot!(n_vals, 10 * n_vals .^ (-2), color=:black, ls=:dash, label=L"$\propto n^{-2}$")
xlabel!(L"n")
plot!(legend=:topright)
title!(latexstring(@sprintf("q = %d", q)))


In [ ]:
q = 2;
n_vals = 5 * 2 .^ (2:1:8);
scatter(n_vals, [abs(ΔE(n_, q) - ΔE_asympt(n_, q)) for n_ in n_vals],
    xscale=:log10, yscale=:log10, label=L"$K^{-1}H_{q+1}$ Error")
scatter!(n_vals, [abs(nC(n_, q) - nC_asympt(n_, q)) for n_ in n_vals], 
    label=L"$nK C(q,n)$ Error", marker=:d)
plot!(n_vals, 100 * n_vals .^ (-2), color=:black, ls=:dash, label=L"$\propto n^{-2}$")
xlabel!(L"n")
plot!(legend=:topright)
title!(latexstring(@sprintf("q = %d", q)))

In [ ]:
q = 3;
n_vals = 5 * 2 .^ (2:1:8);
scatter(n_vals, [abs(ΔE(n_, q) - ΔE_asympt(n_, q)) for n_ in n_vals],
    xscale=:log10, yscale=:log10, label=L"$K^{-1}H_{q+1}$ Error")
scatter!(n_vals, [abs(nC(n_, q) - nC_asympt(n_, q)) for n_ in n_vals],
    label=L"$nK C(q,n)$ Error", marker=:d)
plot!(n_vals, 100 * n_vals .^ (-2), color=:black, ls=:dash, label=L"$\propto n^{-2}$")
xlabel!(L"n")
plot!(legend=:topright)
title!(latexstring(@sprintf("q = %d", q)))